в этом файле - код сбора датасета с портала авто.ру

In [248]:
import json
import ast
import numpy as np
import pandas as pd
DIR_TRAIN = './input/parsing-all-moscow-auto-ru-09-09-2020/'
DIR_TEST = './input/sf-dst-car-price-prediction/'

# тестовая база, в которой надо предсказать колонку цены
test = pd.read_csv(DIR_TEST+'test.csv')

In [336]:
# определение марок автомобилей, по которым надо набрать информацию
brands = test['brand'].value_counts().index.values
brands

array(['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI',
       'MITSUBISHI', 'SKODA', 'VOLVO', 'HONDA', 'INFINITI', 'LEXUS'],
      dtype=object)

In [ ]:
# добыча нужных данных с портала авто.ру -

# код этой ячейки - изучение столбцов исходного датасета test и значений признаков, чтобы найти что-то похожее на сайте авто.ру
# на примере страницы БМВ в ответе браузера ищется json и изучается, какие данные можно оттуда вытянуть
# на html странице оказалось меньше данных можно найти и сложнее их извлекать

# искать в ключах json в т.ч. и имена столбцов, предварительно выбросив некоторые ненужные (image,description)
keystolook = ['bodyType', 'brand', 'color',
              'engineDisplacement', 'enginePower', 'equipment_dict',
              'fuelType',  'mileage', 'modelDate', 'model_info', 'model_name',
              'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
              'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
              'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
              'Привод', 'Руль', 'Состояние', 'Таможня']
valuetolook = []
keyvaluepairtolook = []
for col in keystolook:
    # print(test[col].value_counts()[:2])
    ll = test[test.brand == 'BMW'][col].value_counts(
    ).index[:2].values.tolist()
    if str(ll[0]).__contains__('{'):
        keyvaluepairtolook.extend(ll)  # словари в отдельный список
    else:
        valuetolook.extend(ll)  # значения в другой список

# обработать немного по-другому разветвленный словарь - и его тоже потом разбить на ключи и значения
ll = test[test.brand ==
          'BMW']['complectation_dict'].value_counts().index[:1].values[0]
keyvaluepairtolook.append(ll)

kvpairstofind = []  # преобразование строк в словари
for i in range(len(keyvaluepairtolook)):
    try:
        kvpairstofind.append(ast.literal_eval(keyvaluepairtolook[i]))
    except Exception as ex:
        print(keyvaluepairtolook[i], i, " error:", ex)
# print(kvpairstofind)

tmpkey, tmpval = set(), set()
# разбор словарей отдельно на ключи и значения


def parsedictolists(dic):
    for k, v in dic.items():
        tmpkey.add(k)
        if type(v) == dict:
            parsedictolists(v)
        elif type(v) == list:
            for li in v:
                if type(li) == dict:
                    parsedictolists(li)
                else:
                    tmpval.add(li)
        else:
            tmpval.add(v)


for dic in kvpairstofind:
    parsedictolists(dic)
# формирование списка ключей и списка значений
# этот ключ визуально замечен, тоже добавить для поиска цепочки
keystolook.append('available_options')
tmpkey.update(set(keystolook))
tmpval.update(set(valuetolook))

# здесь собраны неповторяющиеся ключи и значения, которые надо поискать в jsone
print(tmpkey)
print(tmpval)

# две ошибки с распарсиванием значения true пока не рассматривать

In [338]:
from collections import defaultdict
import requests
# данные, просмотренные в мозила- какой заголовок посылать для получения страницы авто.ру
false = False
url = 'https://auto.ru/-/ajax/desktop/listing/'
js = {"category": "cars", "section": "all", "has_image": false, "customs_state_group": "DOESNT_MATTER",
      "year_to": 2020, "page": 1, "catalog_filter": [{"mark": "BMW"}], "geo_radius": 100, "geo_id": [213]}
brheader = '''
Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
x-client-app-version: 8bd42497b89
x-client-date: 1642233376701
x-csrf-token: 9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5
x-requested-with: XMLHttpRequest
x-page-request-id: 18045d227a4fdec2917de7935ca9602e
x-yafp: {"a1":"+9mUFOhH1rWWmw==;0","a2":";1","a3":"RJnfx2MboJswCanp/pTWfA==;2","a4":"StYko3zFWsNlnqVqSmk8wtiCNceQDVXno+Ty8egp/wNyPQ==;3","a5":"wjYu92t/8PFxSg==;4","a6":"Vf8=;5","a7":"KvAfsVXDGAb9HQ==;6","a8":"eShrxqXZZi8JBw==;7","a9":"JW6wMh5dW7W7xg==;8","b1":"WMS1X811zcyfTA==;9","b2":"yyjQa/YhmCMLeQ==;10","b3":"naIppb8qXU+Xlw==;11","b4":"+X3co2U+4Iw=;12","b5":"ogGn4VtDnhQ22Q==;13","b6":"+z2Q4WofX4A=;14","b7":"EoTKve8VDMvjHA==;15","b8":"yUok16zaJal5sA==;16","b9":"siovMEfX8g2Gew==;17","c1":"1mBBIQ==;18","c2":"EdUNHgSksixRjnYOmi68eyuv;19","c3":"db46n2HIzPM1qzgUsfYyHffB;20","c4":"EGxT9UTLOBc=;21","c5":"idSfAP4K5Uo=;22","c6":"FJTeOQ==;23","c7":"tRonCBgZhOs=;24","c8":"M7FsDWGzADg=;25","c9":"pIwxReY43sw=;26","d1":"jPxdojxZZZeSF0iatEsue0X2DnlVMp8dIvaMllbb8LyIXVp/iZrcCU36Bg82evui8HjdaSl/;27","d2":"VDg=;28","d3":"qu1pusEc2NS/fg==;29","d4":"ykBYQZNXFv4=;30","d5":"yxfY97fzUW+BnQ==;31","d7":"RNgWvmQheUM=;32","d8":"JjI+RtXpluifFjkei3ajCpu/QyctL6CvHBo=;33","d9":"r1lNqkxAKho=;34","e1":"d3hBxv/sTectjA==;35","e2":"io2TNyuAibhRog==;36","e3":"dcX6jLdzgCIYhg==;37","e4":"zTY/oGADkWI=;38","e5":"vrzfRKzIg2ZSBA==;39","e6":"6iYhrYEdONs=;40","e7":"U+LLxN5sKjo=;41","e8":"jlUCPDEwNGA=;42","e9":"Insn4K4Iqc4=;43","f1":"r56Ndh8a2QXDkQ==;44","f2":"Qpll3AumPiI=;45","f3":"P5JubqtBinrKGQ==;46","f4":"GB3B60t7OF4=;47","f5":"Mq95O0hQKB2SGQ==;48","f6":"OK73ejIRtckzmA==;49","f7":"C7Jo/l+7Ggjvpw==;50","f8":"BH7thbLCvcJWtQ==;51","f9":"8UPV/s0K/Ak=;52","g1":"jTDuAwUJxtE=;53","g2":"n3Pb4Rhk4qQp5w==;54","g3":"sHw8QnFKhdE=;55","g4":"hanEck+CRe8=;56","g5":"VAJXAcn/KCE=;57","g6":"pMypKwIOiJhyTQ==;58","g7":"CwTU8X2eT3E=;59","g8":"4d26J5yVR4U=;60","g9":"wK7945Zt8i8=;61","h1":"9AInkX3tt0KXng==;62","h2":"l3Rdcw4wZUwafQ==;63","h3":"SR6HwaeL++QM2Q==;64","h4":"l+dC+vblTJZnEg==;65","h5":"UV8OIr2z61s=;66","h6":"X1oSmLCKI52Cfw==;67","h7":"wOjd9QM7NZNcQW/0JjWRD5T52Y3bodsWXoNp2YcH8Wyukhb1UTew/B0cvezKs9QEEap5mU2XIvCJg6uxPf9AsYXo0PUDOyKTCEEt9DM1ig+Q+eGNmqHLFlSDYtmXB6hs55JZ9UY38fwYHLzs37PcBFSqe5lGl2zwnIPlsXT/QbGT6N/1FDsnkx1BbvQiNcIPsPngjd6hxhZUg0TZiwfmbP2SU/VKN6X8Xxw=;68","h8":"/BCuzEtOychW7w==;69","h9":"FBEsYWHV4nvLaw==;70","i1":"AYXrFKDs94o=;71","i2":"Z2r3MleTRcZt9g==;72","i3":"9jhq+iOUK7PxBg==;73","i4":"9HWV3QWKr7TJlg==;74","i5":"UhPNwfrW5YtxyA==;75","z1":"X/S2w8juiivuh5tNnyXFf7ZicLUgsf6fa1fi2Mu2ToIuEE4sm+x+7345sXiWevDxY2Gp6h6fOm/8vWzuIJsEDQ==;76","z2":"89XLvCGRrsGF3dtSHbko1z7GgAO/o/OEW5WlM5ZnexjHx3hvf4rcgDMhYhLUl2gpLH82unkBpFL4IbeSMiQayw==;77","z3":"m9LfDh+/njvJ9w==;78","z6":"+NMrQoLE6YGRZuuA;79","z7":"vpnOcULUNQV8F1xf;80","z8":"Q+PA64DATB+AZg==;81","z9":"dEoDzd8viQ3P3g==;82","y1":"zkSOPfJmbbejBw==;83","y2":"GoY3Mls8TN56Ow==;84","y3":"WnalUUEBHD55VA==;85","y4":"ujyAI5y9XDRYpQ==;86","y5":"8KeCDkNpsaaNEg==;87","y6":"X/StZ62O0x6adQ==;88","y7":"BaDX/sSjTRmNPlJB;89","y8":"5xtR70OjoKUzZQ==;90","y9":"dogOajbNqQSnJg==;91","y10":"urOhyDWLxgQ20g==;92","x1":"5B3uMq7G7GmeIA==;93","x2":"8E18p8rSLsJt+A==;94","x3":"qyIHLcmgjts7Yw==;95","x4":"UqPDrKNBiZA+4A==;96","x5":"q8f9mu/g0BJ1KFS0;97","z5":"MLFA/oKKRWo=;98","z4":"poxmqqYtBdfiNOjClbQ=;99","v":"6.3.1","pgrdt":"tIPfIJoihG27AU6WOtGWU4Ft144=;100","pgrd":"5L+ttwRwCk2f75zf0ZSEywqTgH3k42gYgEvTyb/RL2pTcvJugUYEm/lGL8I60EEVEc9dfWU/bjpewbufCBOR1UgueETv/Kmglh9FrUxQFZwSqgsrZwvtV7LAp/t2eXwVf5dpqPjXBI5fretcUlR+Ah8T5itlg7QXQ7b5UuFTS5/cR43nDhNSvBc/3KMlE5082xCk2gOHclPDJOYEPKyBNznzYqM="}
content-type: application/json
Origin: https://auto.ru
Content-Length: 181
Connection: keep-alive
Cookie: autoru_sid=a%3Ag61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1%7C1642075015356.604800.NyNBXPK57aO3A4-Fo8IjLg.mMqAMysiCxrZlvcJeohRogrbR5jwiGpAsotk9SwB-PI; autoruuid=g61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1; suid=a06c2b573007272e470c6bb6f7511ff2.e5976b34552661d0ab47d79e4fc53c7e; _yasc=JYqgy/CgqEsxvyIjwOjq/i7aIMB4KhoRNSGMgygRRcrkVt4/ELo=; yuidlt=1; yandexuid=3644335621586616841; yandex_login=; mda2_beacon=1642093553695; crookie=ZZh3+UR44UJlHEcXIHCHEhLtuL1SWoMmxMgXvbZOjEKfcgYC+Qfo8QOA7oa+xrYCTmpAkGV1DUCRG8wwHzaMSsouZ9M=; cmtchd=MTY0MjA3NTAyNDQ0Nw==; gids=213; gradius=100; los=1; bltsr=1; _csrf_token=9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5; from_lifetime=1642233370088; from=direct; ys=c_chck.1432299411; autoru-visits-count=1; X-Vertis-DC=vla
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: same-origin
Sec-Fetch-Site: same-origin
'''
# подготовка заголовка headers для запроса requests- сформировать словарь,
# из заголовка выброшены пара строк с упоминанием page=... и результаты не изменились.
# так будет проще посылать запрос на страницу только по page в словаре json=js
brheader = brheader.split('\n')
brheaderdict = {}
for item in brheader:
    if not(item):
        continue
    kv = item.split(': ')
    key, value = kv[0], kv[1]
    brheaderdict[key] = value
pass

# эта функция вызывается позже для прохода по всем маркам и всем страницам для каждой марки на портале авто.ру
# и распарсивание каждого json ответа


def responsedic(page, js, mark) -> dict:
    js['page'] = page
    js['catalog_filter'] = [{"mark": mark}]
    response = requests.post(url, json=js, headers=brheaderdict)
    if response.status_code == 200:
        return defaultdict(lambda: "нет в jsone", response.json())

# datadic=responsedic(1,js)
# with open("temp.json","w") as f:
#     json.dump(datadic,f)

In [199]:
# изучение ответного json с помощью вьюеров и notepad++


# проба - поправить куски текста с авто.ру, которые имеют нарушенную структуру
# встречались несбалансированные слэши и jsonviewer не просматривает эти данные
# import json
# import re

# class LazyDecoder(json.JSONDecoder):
#     def decode(self, s, **kwargs):
#         regex_replacements = [
#             (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
#             (re.compile(r',(\s*])'), r'\1'),
#         ]
#         for regex, replacement in regex_replacements:
#             s = regex.sub(replacement, s)
#         return super().decode(s, **kwargs)

# другой вариант просмотреть json, когда в возвращаемых данных json есть некорректные данные
# используя итератор для зеркального словаря datadic, удаление из datadic_ больших кусков текста,
# в них попадаются несбалансированные кавычки, бэкслэши и хрен знает что еще и json их не открывает
import copy

datadic = responsedic(1, js, 'BMW')

datadic_ = copy.deepcopy(datadic)


def parsedictremovechunk(mirrordic, dic, isfirst=True):
    for k, v in mirrordic.items():
        if type(v) == dict:
            parsedictremovechunk(v, dic[k], isfirst)
        elif type(v) == list:
            curind = 0
            for li in v:
                if type(li) == dict:
                    parsedictremovechunk(li, dic[k][curind], isfirst)
                    curind += 1
        # если большой кусок текста - удалить его, он все равно не понадобится для предиктов
        elif type(v) == str and len(v) > 150:
            if isfirst:
                if dic[k] == v:
                    dic[k] = "text removed"
                else:
                    print("не зеркало")
            else:
                print("проверка {},{}\n{}".format(k, dic[k], v))
        else:
            pass
            # print("неожиданный тип",type(v))


parsedictremovechunk(datadic, datadic_)
# проверка
parsedictremovechunk(datadic, datadic_, False)


# еще вариант
# попытки прочитать в json словарь из подготовленной строки
# tmp=datadic_.__str__()
# tmp=tmp.replace("'",'"')
# tmp=tmp.replace("False",'false')
# tmp=tmp.replace("True",'true')
# tmp=tmp.replace("None",'null')
# js=json.loads(tmp)
# js=json.loads(tmp, cls=LazyDecoder)
# datadic_=ast.literal_eval(tmp)
# tmp=json.dumps(datadic_)
# tmp

In [215]:
# исследование возвращаемого json
# просто в дополнение к визуальному поиску - машинный поиск разных ключей и значений


# задача - найти нужные данные в json, возвращаемом с портала авто.ру, чтобы понимать, откуда их понадергать для нового датасета
# анализ проводить по данным для БМВ - это не сбор данных, а только поиск, откуда вытягивать
# проанализировать возвращаемый посредством post кусок данных json на включение требуемых признаков из тестового датасета test
# datadic -словарь ответ json портала авто.ру
# debugpath цепочка запроса к datadic, где нашлось что-то похожее
# rootkey - для пропуска всех корневых ключей и просмотра только списка офферов
import time


def deepintodict(datadic, debugpath, rootkey="") -> str:
    def comparek(keystofind, key, value, debugpath) -> str:
        for keytofind in keystofind:
            if keytofind == key:
                existkey.add(keytofind)
                if not deepkeys.__contains__(debugpath):
                    deepkeys.append(debugpath)
                    print(debugpath+"='{}'".format(value))
                    return "['{}']='{}'".format(key, value)
        return ''

    def comparev(valuestofind, key, value, debugpath) -> str:
        for valuetofind in valuestofind:
            if valuetofind == value:
                existval.add(valuetofind)
                if not deepkeys.__contains__(debugpath):
                    deepkeys.append(debugpath)
                    print(debugpath+"='{}'".format(value))
                    return "['{}']='{}'".format(key, value)
        return ''

    res1, res = "", ""
    try:
        for key, value in datadic.items():
            if rootkey and rootkey != key:
                continue  # рассмотреть только один offers - из корневых ключей
            td = type(value)
            debugpath_ = debugpath+"['{}']".format(key)
            if td == dict:
                res1 = comparek(keystofind, key, value, debugpath_)
                res = deepintodict(value, debugpath_)
                if res:
                    res1 = "['{}']{}".format(key, res)
                    pass
                continue
            if td == list:
                # если список словарей, то пройти по словарям
                if len(value) > 0 and type(value[0]) == dict:
                    count = 0
                    for item in value:
                        res = deepintodict(
                            item, "{}[{}]".format(debugpath_, 'x'))
                        if res:
                            res1 = "['{}'][{}]{}".format(key, count, res)
                            pass
                        count += 1
                        continue
                else:  # если список значений один ключ key
                    res1 = comparek(keystofind, key, value, debugpath_)
                    for valinlist in value:
                        res1 = comparev(valuestofind, key,
                                        valinlist, debugpath_)
            else:
                res1 = comparek(keystofind, key, value, debugpath_)
                res1 = comparev(valuestofind, key, value, debugpath_)
    except Exception as ex:
        print(res1, res, ": здесь ошибка {0} ".format(ex))
    return res1


# сюда сохранять ключи и значения, которые нашлись в jsone
existkey = set()
existval = set()
# рабочий список для хранения цепочки ключей без учета индексов (индексы заменяются на х) - чтобы просто найти хоть одну цепочку ключей и понять, нужно это или нет
deepkeys = []
# взять нужные столбцы из test и поискать совпадения, а так же добавить пару ключей для больших списков разных опций, которые визуально выявлены при просмотре датасета test
keystofind = list(tmpkey)
# взять наиболее часто встречающиеся значения из датасета test
valuestofind = list(tmpval)
t0 = 0
for page in range(1, 10):
    # получить с сервера страницу page в словарь
    while time.time_ns() - t0 < 500000000:  # проверять время, чтобы не получить отказ от сервера
        time.sleep(0.1)
    datadic = responsedic(page, js, 'BMW')
    t0 = time.time_ns()  # начальное время старта
    # найти в словаре ключи и значения и соответствующие им полные цепочки ключей, где поискать нужные данные в этом jsone
    res = deepintodict(datadic, "", rootkey='offers')
    # теперь пополнились множества existkey,existval
    # определить, что нашлось и что- нет
    difk = set(keystofind).difference(existkey)
    difv = set(valuestofind).difference(existval)
    print("\nне найдены ключи {}из{}".format(len(difk), len(keystofind)), difk)
    print("не найдены значения {}из{}".format(
        len(difv), len(valuestofind)), difv)
    print("\n\n")

    # вызвать следующую страницу с авто.ру и поискать оставшиеся ключи из начения там
    # на следующей странице искать не все, а среди оставшихся
    keystofind = list(difk)
    valuestofind = list(difv)

['offers'][x]['owner_expenses']['transport_tax']['horse_power']='249'
['offers'][x]['additional_info']['days_in_stock']='4'
['offers'][x]['additional_info']['days_on_sale']='4'
['offers'][x]['id']='1106507931'
['offers'][x]['seller']['location']['region_info']['id']='213'
['offers'][x]['seller']['location']['region_info']['name']='Москва'
['offers'][x]['seller']['location']['timezone_info']['name']='Europe/Moscow'
['offers'][x]['seller']['name']='id13039842'
['offers'][x]['state']['mileage']='51000'
['offers'][x]['state']['external_panorama']['published']['id']='1419387375-1641654220784-sDduG'
['offers'][x]['state']['image_urls'][x]['name']='1'
['offers'][x]['vehicle_info']['configuration']['id']='20158770'
['offers'][x]['vehicle_info']['configuration']['doors_count']='5'
['offers'][x]['vehicle_info']['configuration']['human_name']='Внедорожник 5 дв.'
['offers'][x]['vehicle_info']['complectation']['id']='21075655'
['offers'][x]['vehicle_info']['complectation']['name']='xDrive30d M Spor

In [227]:
# далее изучаем json и собираем для столбцов ключи:

# получился такой набор - который можно преобразовать в базу данных , идентичную по наполнению исходной test.csv

# bodyType
# ['offers'][x]['vehicle_info']['configuration']['human_name']='Универсал 5 дв.'
# перевести в нижний регистр
# brand
# ['offers'][x]['vehicle_info']['mark_info']['code']='BMW'
# ['offers'][x]['vehicle_info']['mark_info']['name']='BMW'
# color ???? переводить как-то
# ['color_hex']
# ['offers'][x]['vehicle_info']['complectation']['vendor_colors'][x]['stock_color']['name_ru']='белый'
# complectation_dict
# ['offers'][x]['vehicle_info']['complectation']
# ['offers'][x]['vehicle_info']['complectation']['available_options']='['airbag-passenger', 'lock', 'electro-mirrors', 'mirrors-heat', 'seat-transformation', 'wheel-power', 'fabric-seats', 'abs', 'wheel-leather', 'tinted-glass', 'audiopreparation', 'ashtray-and-cigarette-lighter', 'front-centre-armrest', 'body-mouldings', '15-inch-wheels', 'airbag-driver', 'isofix', 'electro-window-front', 'paint-metallic', 'ptf', 'migration-flag', 'power-child-locks-rear-doors', 'wheel-configuration2', 'wheel-configuration1', 'immo']'

# engineDisplacement (2.0 LTR)
# #только литраж найден в компоновочной записи (как 2.0d соответствует литражу)
# ['offers'][x]['vehicle_info']['lk_summary']='"2.0d AT (150 л.с.), внедорожник, полный, дизель"'
# lk_summary иногда не возвращается

# enginePower (249 N12- надо еще N12 пристегнуть для однотипности с датасетом test)
# ['offers'][x]['owner_expenses']['transport_tax']['horse_power']='249'
# ['offers'][x]['vehicle_info']['tech_param']['power']='249'
# ['offers'][x]['vehicle_info']['tech_param']['power_kvt']='183'

# equipment_dict
# # найдено визуально в json
# ['offers'][x]['vehicle_info']['equipment']
# fuelType (в датасете по русски)
# в jsone по-русски толко в lk_summary
# ['offers'][x]['vehicle_info']['tech_param']['engine_type']='DIESEL'

# mileage
# ['offers'][x]['state']['mileage']='51000'

# productionDate
# ['offers'][x]['documents']['year']='2009'

# modelDate не нашел, что это за дата
# ['offers'][x]['vehicle_info']['super_gen']['year_from']='2013'
# ['offers'][x]['vehicle_info']['super_gen']['year_to']='2012'

# model_info
# ['offers'][x]['vehicle_info']['model_info']='{'code': 'X6', 'name': 'X6', 'ru_name': 'Х6', 'morphology': {}, 'nameplate': {'code': '9265282', 'name': '30d', 'semantic_url': '30d'}}'

# model_name
# ['offers'][x]['vehicle_info']['model_info']['name']

# name 1.6 AT (110 л.с.) надо как-то клеить
# ['offers'][x]['vehicle_info']['tech_param']['human_name']='30d 3.0d AT (249 л.с.) 4WD'

# numberOfDoors
# ['offers'][x]['vehicle_info']['configuration']['doors_count']='5'

# parsing_unixtime
# в jsone не нашел ничего похожего

# super_gen словарь
# ['offers'][x]['vehicle_info']['tech_param']

# vehicleConfiguration
# похоже на склейку трех параметров
# ['offers'][x]['vehicle_info']['configuration']['body_type']
# ['offers'][x]['vehicle_info']['tech_param']['transmission']='AUTOMATIC'
# и литраж надо прицепить сюда же

# vehicleTransmission русских атрибутов в jsone не нашлось, надо сопоставлять
# ['offers'][x]['vehicle_info']['tech_param']['transmission']='AUTOMATIC' автоматическая

# vendor
# ['offers'][x]['vehicle_info']['vendor']='EUROPEAN'

# Владельцы 1 владелец,2 владельца,3 или более
# ['offers'][x]['documents']['owners_number']='4' надо допилить

# Владение
# ['offers'][x]['owner_expences']['holding_period_month']=12 везде одинаково вроде
# значит вычислять как то

# ПТС СОПОСТАВЛЯТЬ    оРИГИНАЛ дУБЛИКАТ
# ['offers'][x]['documents']['pts_original']=true
# ['offers'][x]['documents']['pts']=ORIGINAL

# Привод Передний,Полный,Задний сопоставлять
# ['offers'][x]['vehicle_info']['tech_param']['gear_type']='ALL_WHEEL_DRIVE'
# ['offers'][x]['vehicle_info']['tech_param']['gear_type']='REAR_DRIVE'

# Руль Левый Правый -сопоставлять
# ['offers'][x]['vehicle_info']['steering_wheel']=LEFT

In [630]:
# сбор данных именно в таком виде, как они представлены в test датасете
import time
from collections import defaultdict


class default0(dict):
    def __missing__(self, key):
        return key.__str__()+'не найден'
# заполнить датасет из словаря, полученного с сервера


def feelDF(dic, df, mark, istest=False) -> pd.DataFrame:
    tst = test[test.brand == mark]
    # словари для сопоставления в основном английских и русских наименований, когда в ответе jsone не нашлось ничего подходящего (а в html почему-то есть)
    Wheel = defaultdict(lambda: "absent", {'LEFT': 'Левый', 'RIGHT': 'Правый'})
    Privod = default0({'ALL_WHEEL_DRIVE': 'полный',
                      'REAR_DRIVE': 'задний', 'FORWARD_CONTROL': 'передний'})
    Pts = default0({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'})
    Owner = default0({1: '1 владелец', 2: '2 владельца', 3: '3 или более',
                     4: '3 или более', 5: '3 или более', 6: '3 или более'})
    # настоящий гиморой найти сопоставление для кодов цветов - в jsone только коды.
    color = defaultdict(lambda: "другой цвет", {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий',
                                                'EE1D19': 'красный', '': 'коричневый', '': 'зелёный', '': 'бежевый', '': 'голубой', '': 'золотистый',
                                                '': 'пурпурный', '': 'фиолетовый', '': 'жёлтый', '': 'оранжевый', '': 'розовый'})
    fuelType = defaultdict(lambda: "другое топливо", {
                           'DIESEL': 'дизель', 'GASOLINE': 'бензин', 'HYBRID': 'гибрид'})
    vehicleTransmission = default0(
        {'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая', 'VARIATOR': 'вариатор', 'ROBOT': 'роботизированная'})
    for offerInd in range(len(dic['offers'])):
        dto = defaultdict(lambda: "keyinoffer absent", dic['offers'][offerInd])
        # иногда такого нет в jsone, чтобы не обрывался процесс сбора
        dto_vehicle_info = defaultdict(
            lambda: "keyinvehicle_info absent!", dto['vehicle_info'])
        iscomplectation = dto_vehicle_info['complectation']['id'] != '0'
        if iscomplectation and 'vendor_colors' in dto_vehicle_info['complectation']:
            dto_vehicle_info['complectation'].pop('vendor_colors')

        isequipment_dict = len(dto_vehicle_info['equipment']) != 0
        isowner_expenses_dict = dto['owner_expenses'] != "keyinoffer absent" and (
            'transport_tax' in dto['owner_expenses']) and ('holding_period_month' in dto['owner_expenses']['transport_tax'])
        isprice = ('price_info' in dto) \
            and ('price' in dto['price_info'])
        if not isprice:
            continue
        if not ('owners_number' in dto['documents']):
            continue  # в тестовой базе нет машин без владельцев, значит такие не нужны для тренировочных данных

        # словарь формируется из исходного датасета
        origdic = {'bodyType_': tst['bodyType'].iloc[offerInd],
                   'brand_': tst['brand'].iloc[offerInd],
                   'color_': tst['color'].iloc[offerInd],
                   'complectation_dict_': tst['complectation_dict'].iloc[offerInd],
                   'engineDisplacement_': tst['engineDisplacement'].iloc[offerInd],
                   'enginePower_': tst['enginePower'].iloc[offerInd],
                   'equipment_dict_': tst['equipment_dict'].iloc[offerInd],
                   'fuelType_': tst['fuelType'].iloc[offerInd],
                   'mileage_': tst['mileage'].iloc[offerInd],
                   'productionDate_': tst['productionDate'].iloc[offerInd],
                   'sell_id_': tst['sell_id'].iloc[offerInd],
                   'modelDate_': tst['modelDate'].iloc[offerInd],
                   'model_info_': tst['model_info'].iloc[offerInd],
                   'model_name_': tst['model_name'].iloc[offerInd],
                   'name_': tst['name'].iloc[offerInd],
                   'numberOfDoors_': tst['numberOfDoors'].iloc[offerInd],
                   'parsing_unixtime_': tst['parsing_unixtime'].iloc[offerInd],
                   'super_gen_': tst['super_gen'].iloc[offerInd],
                   'vehicleConfiguration_': tst['vehicleConfiguration'].iloc[offerInd],
                   'vehicleTransmission_': tst['vehicleTransmission'].iloc[offerInd],
                   'vendor_': tst['vendor'].iloc[offerInd],
                   'Владельцы_': tst['Владельцы'].iloc[offerInd],
                   'Владение_': tst['Владение'].iloc[offerInd],
                   'ПТС_': tst['ПТС'].iloc[offerInd],
                   'Привод_': tst['Привод'].iloc[offerInd],
                   'Руль_': tst['Руль'].iloc[offerInd],
                   }

        # новый словарь - данные вытягиваются с портала авто.ру
        _row = {'bodyType': dto_vehicle_info['configuration']['human_name'].lower(),
                # для мерседеса прибавляется бенц - это мне не надо
                'brand': dto_vehicle_info['mark_info']['name'].upper() if mark == dto_vehicle_info['mark_info']['name'].upper() else mark,
                'color': color[dto['color_hex']],
                'complectation_dict': dto_vehicle_info['complectation'].__str__().replace("'", '"') if iscomplectation else np.NaN,

                'engineDisplacement': dto['lk_summary'][:3]+" LTR",

                'enginePower': dto_vehicle_info['tech_param']['power'].__str__()+" N12",

                'equipment_dict': dto_vehicle_info['equipment'].__str__().replace("'", '"') if isequipment_dict else np.NaN,
                'fuelType': fuelType[dto_vehicle_info['tech_param']['engine_type']],
                'mileage': dto['state']['mileage'],

                'productionDate': dto['documents']['year'],
                'sell_id': dto['saleId'].split('-')[0] if 'saleId' in dto else np.NaN,
                'modelDate': dto_vehicle_info['super_gen']['year_from'],
                'model_info': dto_vehicle_info['model_info'].__str__().replace("'", '"'),

                'model_name': dto_vehicle_info['model_info']['code'],
                'name': dto_vehicle_info['tech_param']['human_name'],
                'numberOfDoors': dto_vehicle_info['configuration']['doors_count'],

                # не понял, что это за признак, очень похоже на отметку, когда парсились данные
                # перевод в дату показывает 1970год - или ошибка, или этот признак никак нельзя использовать
                # датапрофайлер показывает высокую корелляцию с brand,engineDisplacement,vendor .....
                # в датасете test признак имеет 9 значащих цифр, а в базе авто.ру 12. разделить на тыщу, чтобы порядок был похож хотябы.
                # это похоже на время вывода объявления - тоже бесполезная информация
                'parsing_unixtime': int(dto['additional_info']['creation_date'])/1000,

                'super_gen': dto_vehicle_info['tech_param'].__str__().replace("'", '"'),
                'vehicleConfiguration': '{} {} {}'.format(dto_vehicle_info['configuration']['body_type'], dto_vehicle_info['tech_param']['transmission'], dto['lk_summary'][:3]),
                'vehicleTransmission': vehicleTransmission[dto_vehicle_info['tech_param']['transmission']],

                'vendor': dto_vehicle_info['vendor'],
                'Владельцы': Owner[dto['documents']['owners_number']],

                # мусор - всегда NaN или 12
                'Владение': dto['owner_expenses']['transport_tax']['holding_period_month'] if isowner_expenses_dict else np.NaN,

                'ПТС': Pts[dto['documents']['pts']],
                'Привод': Privod[dto_vehicle_info['tech_param']['gear_type']],
                'Руль': Wheel[dto_vehicle_info['steering_wheel']],
                'price': dto['price_info']['price'],
                # сохранить цены в валюте на всякий случай
                'priceEUR': dto['price_info']['EUR'],
                'priceUSD': dto['price_info']['USD'],
                }
        new_row = {}
        ri = list(_row.items())
        # insert = lambda _dict, obj, pos: {k: v for k, v in (list(_dict.items())[:pos] + list(obj.items()) + list(_dict.items())[pos:])}
        for i in range(len(_row)):
            new_row.update([ri[i]])
            if istest:  # пока изучаем, как набирается база, ставить столбцы из исходного датасета рядом с комплектуемыми и сравнивать визуально
                if i < len(origdic):
                    # тут нету признаков цены, этот словарь короче
                    new_row.update([list(origdic.items())[i]])

        df = df.append(new_row, ignore_index=True)
    return df

In [574]:
# по каждой марке берем выборочный запрос и ниже смотрим - насколько правильно комплектуется новый датафрейм
# итеративно поправляем косяки
df = pd.DataFrame()
t0 = 0
datadic = {}
marks = ['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI',
         'MITSUBISHI', 'SKODA', 'VOLVO', 'HONDA', 'INFINITI', 'LEXUS']
mark = 'LEXUS'
for page in range(1, 2):
    # получить с сервера страницу page в словарь
    while time.time_ns() - t0 < 500000000:  # проверять время, чтобы не получить отказ от сервера
        time.sleep(0.1)
    # получаем ответ с сервера авто.ру
    datadic = responsedic(page, js, mark)
    t0 = time.time_ns()  # начальное время старта
    # заполняем мою базу
    df = feelDF(datadic, df, mark, istest=True)
    pass

In [ ]:
# попытки разобраться с признаком parse_unixtime


# from datetime import datetime
# def parseunix(x):
#     ts = int(x)/1000
#     return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
# # if you encounter a "year is out of range" error the timestamp
# # may be in milliseconds, try `ts /= 1000` in that case

# df_=df.loc[:,['color','color_','Владение','Владение_','parsing_unixtime','parsing_unixtime_']]
# df_['unixtime']=df['parsing_unixtime'].apply(parseunix)
# df_['unixtime_']=df['parsing_unixtime_'].apply(parseunix)

# df_.sort_values(by='unixtime_',ascending=True)

In [575]:
# дальше выводим сравнение исходного датасета с комплектуемым вручную и смотрим, что не так
import operator

# vals = ['бензин','дизель']
# ops = [np.logical_not, np.logical_not]
# m = np.logical_and.reduce([op(df['fuelType'], v) for op, v in zip(ops, vals)])
# сравнение вытянутых данных и исходных столбцов и итеративные исправления feelDF
# problemcols=['color','color_','Владение','Владение_']
# df.loc[:,['color','color_','Владение','Владение_','parsing_unixtime','parsing_unixtime_']]
# df.sample(5).iloc[:,:] # & df.vehicleTransmission.ne('автоматическая').[df.Владение.notna()]df.vehicleTransmission.ne('автоматическая') & df.vehicleTransmission.ne('механическая')
display(df.sample(2).iloc[:, :10])
display(df.sample(2).iloc[:, 10:20])
display(df.sample(2).iloc[:, 20:30])
display(df.sample(2).iloc[:, 30:])
# display(df.sample(2).iloc[:,38:])

# после проверки по всем маркам - набираемая база будет почти соответствовать по информации исходному датасету test
# не хватило времени выявить все коды цветов и разобраться с parse_unixtime и владение
# признак владение  - придется выкинуть из базы при обучении

,bodyType,bodyType_,brand,brand_,color,color_,complectation_dict,complectation_dict_,engineDisplacement,engineDisplacement_
15,внедорожник 5 дв.,внедорожник 5 дв.,LEXUS,LEXUS,синий,белый,NaN,NaN,2.0 LTR,3.5 LTR
31,внедорожник 5 дв.,внедорожник 5 дв.,LEXUS,LEXUS,белый,белый,NaN,NaN,4.5 LTR,2.7 LTR


,enginePower,enginePower_,equipment_dict,equipment_dict_,fuelType,fuelType_,mileage,mileage_,productionDate,productionDate_
15,150 N12,277 N12,"{""asr"": True, ""esp"": True, ""airbag-driver"": Tr...","{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,бензин,106000.0,55000.0,2016.0,2008.0
23,215 N12,184 N12,"{""cruise-control"": True, ""asr"": True, ""esp"": T...","{""cruise-control"":true,""front-seats-heat-vent""...",бензин,бензин,168000.0,66505.0,2008.0,2016.0


,sell_id,sell_id_,modelDate,modelDate_,model_info,model_info_,model_name,model_name_,name,name_
4,1106597736,1.096755e+09,1997.0,2016.0,"{""code"": ""RX"", ""name"": ""RX"", ""ru_name"": ""RX""}","{""code"":""IS"",""name"":""IS"",""ru_name"":""IS"",""morph...",RX,IS,300 3.0 AT (201 л.с.) 4WD,300 2.0 AT (245 л.с.)
31,1106039880,1.101311e+09,2015.0,2012.0,"{""code"": ""LX"", ""name"": ""LX"", ""ru_name"": ""LX""}","{""code"":""RX"",""name"":""RX"",""ru_name"":""RX"",""morph...",LX,RX,450d 4.5d AT (272 л.с.) 4WD,270 2.7 AT (188 л.с.)


,numberOfDoors,numberOfDoors_,parsing_unixtime,parsing_unixtime_,super_gen,super_gen_,vehicleConfiguration,vehicleConfiguration_,vehicleTransmission,vehicleTransmission_,...,Владение_,ПТС,ПТС_,Привод,Привод_,Руль,Руль_,price,priceEUR,priceUSD
33,5.0,5.0,1.642349e+09,1.603299e+09,"{""gear_type"": ""ALL_WHEEL_DRIVE"", ""engine_type""...","{""id"":""6415529"",""name"":""200h"",""nameplate"":""200...",ALLROAD_5_DOORS AUTOMATIC 5.7,HATCHBACK_5_DOORS VARIATOR 1.8,автоматическая,вариатор,...,NaN,Оригинал,Оригинал,полный,передний,Левый,Левый,3340000.0,39729.0,44957.0
18,5.0,4.0,1.642406e+09,1.603299e+09,"{""gear_type"": ""FORWARD_CONTROL"", ""engine_type""...","{""id"":""20393536"",""name"":""300"",""nameplate"":""300...",ALLROAD_5_DOORS AUTOMATIC 3.5,SEDAN AUTOMATIC 3.0,автоматическая,автоматическая,...,NaN,Оригинал,Оригинал,передний,задний,Левый,Левый,3930000.0,46747.0,52898.0


In [631]:
# проверка выдергивания нужных данных из авто.ру и сохранение
df = pd.DataFrame()
t0 = 0
datadic = {}
numpages = 99
# numpages=2

# для каждой марки автомобиля
# с фильтром в запросе Москва+100км, объявления до 2020г, по всем страницам пройти и записать в файлы информацию
for mark in marks[:]:

    for page in range(1, numpages+1):
        df = pd.DataFrame()  # для каждой марки отдельный файл постранично
        # получить с сервера страницу page в словарь
        while time.time_ns() - t0 < 500000000:  # проверять время, чтобы не получить отказ от сервера
            time.sleep(0.1)
        datadic = responsedic(page, js, mark)
        t0 = time.time_ns()  # начальное время старта
        # "pagination":{"total_page_count":99,"total_offers_count":4301,"page":2,"page_size":37,"from":38,"to":74,"current":2}
        pg = datadic['pagination']
        print("from {} to {} ({}) totoffers {} totpage {} ->".format(
            pg["from"], pg["to"], pg["page_size"], pg['total_offers_count'], pg['total_page_count']), end=" ...")
        df = feelDF(datadic, df, mark)
        fname = 'train/{0}{1:03d}.csv'.format(mark, page)
        print("save ", fname, end=" ...")
        df.to_csv(fname, index=False)
        print(" done")
        totpage = pg['total_page_count']
        if page < totpage:
            continue
        else:
            print("no more pages for ", mark)
            break

# test['bodyType'].value_counts().index.values

# tmp=json.dumps(datadic)
# tmp

from 1 to 37 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI001.csv ... done
from 38 to 74 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI002.csv ... done
from 75 to 111 (37) totoffers 1535 totpage 42 -> ...save  train/MITSUBISHI003.csv ... done
from 112 to 148 (37) totoffers 1535 totpage 42 -> ...save  train/MITSUBISHI004.csv ... done
from 149 to 185 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI005.csv ... done
from 186 to 222 (37) totoffers 1535 totpage 42 -> ...save  train/MITSUBISHI006.csv ... done
from 223 to 259 (37) totoffers 1535 totpage 42 -> ...save  train/MITSUBISHI007.csv ... done
from 260 to 296 (37) totoffers 1535 totpage 42 -> ...save  train/MITSUBISHI008.csv ... done
from 297 to 333 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI009.csv ... done
from 334 to 370 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI010.csv ... done
from 371 to 407 (37) totoffers 1534 totpage 42 -> ...save  train/MITSUBISHI011.csv ...

train [] ['AUDI001.csv', 'AUDI002.csv', 'AUDI003.csv', 'AUDI004.csv', 'AUDI005.csv', 'AUDI006.csv', 'AUDI007.csv', 'AUDI008.csv', 'AUDI009.csv', 'AUDI010.csv', 'AUDI011.csv', 'AUDI012.csv', 'AUDI013.csv', 'AUDI014.csv', 'AUDI015.csv', 'AUDI016.csv', 'AUDI017.csv', 'AUDI018.csv', 'AUDI019.csv', 'AUDI020.csv', 'AUDI021.csv', 'AUDI022.csv', 'AUDI023.csv', 'AUDI024.csv', 'AUDI025.csv', 'AUDI026.csv', 'AUDI027.csv', 'AUDI028.csv', 'AUDI029.csv', 'AUDI030.csv', 'AUDI031.csv', 'AUDI032.csv', 'AUDI033.csv', 'AUDI034.csv', 'AUDI035.csv', 'AUDI036.csv', 'AUDI037.csv', 'AUDI038.csv', 'AUDI039.csv', 'AUDI040.csv', 'AUDI041.csv', 'AUDI042.csv', 'AUDI043.csv', 'AUDI044.csv', 'AUDI045.csv', 'AUDI046.csv', 'AUDI047.csv', 'AUDI048.csv', 'AUDI049.csv', 'AUDI050.csv', 'AUDI051.csv', 'AUDI052.csv', 'AUDI053.csv', 'AUDI054.csv', 'AUDI055.csv', 'AUDI056.csv', 'AUDI057.csv', 'AUDI058.csv', 'BMW001.csv', 'BMW002.csv', 'BMW003.csv', 'BMW004.csv', 'BMW005.csv', 'BMW006.csv', 'BMW007.csv', 'BMW008.csv', 'BMW009.

In [634]:
# проверка считывания и набора тренировочной базы
import os
for root, dirs, files in os.walk('train'):
    print(root, dirs, files)
trainfname = 'dftrain.csv'
dftest = pd.DataFrame()
for fname in files:
    try:
        print("read ", fname, end=" ...")
        df_ = pd.read_csv('train/{0}'.format(fname))
        print(" concat", end=" ...")
        dftest = pd.concat([dftest, df_])
        print(" done")
    except Exception as ex:
        print(ex)

dftest.to_csv(trainfname, index=False)  # сохранение итоговой базы

train [] ['AUDI001.csv', 'AUDI002.csv', 'AUDI003.csv', 'AUDI004.csv', 'AUDI005.csv', 'AUDI006.csv', 'AUDI007.csv', 'AUDI008.csv', 'AUDI009.csv', 'AUDI010.csv', 'AUDI011.csv', 'AUDI012.csv', 'AUDI013.csv', 'AUDI014.csv', 'AUDI015.csv', 'AUDI016.csv', 'AUDI017.csv', 'AUDI018.csv', 'AUDI019.csv', 'AUDI020.csv', 'AUDI021.csv', 'AUDI022.csv', 'AUDI023.csv', 'AUDI024.csv', 'AUDI025.csv', 'AUDI026.csv', 'AUDI027.csv', 'AUDI028.csv', 'AUDI029.csv', 'AUDI030.csv', 'AUDI031.csv', 'AUDI032.csv', 'AUDI033.csv', 'AUDI034.csv', 'AUDI035.csv', 'AUDI036.csv', 'AUDI037.csv', 'AUDI038.csv', 'AUDI039.csv', 'AUDI040.csv', 'AUDI041.csv', 'AUDI042.csv', 'AUDI043.csv', 'AUDI044.csv', 'AUDI045.csv', 'AUDI046.csv', 'AUDI047.csv', 'AUDI048.csv', 'AUDI049.csv', 'AUDI050.csv', 'AUDI051.csv', 'AUDI052.csv', 'AUDI053.csv', 'AUDI054.csv', 'AUDI055.csv', 'AUDI056.csv', 'AUDI057.csv', 'AUDI058.csv', 'BMW001.csv', 'BMW002.csv', 'BMW003.csv', 'BMW004.csv', 'BMW005.csv', 'BMW006.csv', 'BMW007.csv', 'BMW008.csv', 'BMW009.


# тренировочная база dftrain.csv готова - 5 дней работы


In [635]:
dftest = pd.read_csv(trainfname)
dftest.sample(3).iloc[:, 10:]

,sell_id,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,price,priceEUR,priceUSD
24858,1106481984,2008.0,"{""code"": ""XC60"", ""name"": ""XC60"", ""ru_name"": ""X...",XC60,2.0 AMT (203 л.с.),5.0,1.641306e+09,"{""gear_type"": ""FORWARD_CONTROL"", ""engine_type""...",ALLROAD_5_DOORS ROBOT 2.0,роботизированная,EUROPEAN,3 или более,12.0,Дубликат,передний,Левый,1200000.0,14274.0,16152.0
9098,1106069659,2014.0,"{""code"": ""V_KLASSE"", ""name"": ""V-Класс"", ""ru_na...",V_KLASSE,220 d экстра длинный 2.1d AT (163 л.с.) 4WD,5.0,1.640870e+09,"{""gear_type"": ""ALL_WHEEL_DRIVE"", ""engine_type""...",MINIVAN AUTOMATIC 2.1,автоматическая,EUROPEAN,2 владельца,12.0,Оригинал,полный,Левый,13340000.0,158678.0,179560.0
24514,1106481747,2007.0,"{""code"": ""S40"", ""name"": ""S40"", ""ru_name"": ""S40""}",S40,2.5 MT (230 л.с.) 4WD,4.0,1.641302e+09,"{""gear_type"": ""ALL_WHEEL_DRIVE"", ""engine_type""...",SEDAN MECHANICAL 2.5,механическая,EUROPEAN,1 владелец,12.0,Оригинал,полный,Левый,810000.0,9635.0,10903.0


In [636]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25099 entries, 0 to 25098
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              25099 non-null  object 
 1   brand                 25099 non-null  object 
 2   color                 25099 non-null  object 
 3   complectation_dict    7654 non-null   object 
 4   engineDisplacement    25099 non-null  object 
 5   enginePower           25099 non-null  object 
 6   equipment_dict        20247 non-null  object 
 7   fuelType              25099 non-null  object 
 8   mileage               25099 non-null  float64
 9   productionDate        25099 non-null  float64
 10  sell_id               25099 non-null  int64  
 11  modelDate             25099 non-null  float64
 12  model_info            25099 non-null  object 
 13  model_name            25099 non-null  object 
 14  name                  25099 non-null  object 
 15  numberOfDoors      

In [629]:
from datetime import datetime
dftest.sample(3).iloc[:, 10:]


def parseunix(x):
    ts = int(x)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')


df_ = dftest.sample(3).loc[:, ['parsing_unixtime']]
df_['unixtime'] = df_['parsing_unixtime'].apply(parseunix)

df_.sort_values(by='unixtime', ascending=True)

,parsing_unixtime,unixtime
520,1.641846e+09,2022-01-10
840,1.642073e+09,2022-01-13
52,1.642141e+09,2022-01-14
